In [1]:
import os
import logging
import pandas as pd
import numpy as np
import SimpleITK as sitk
import radiomics
from radiomics import featureextractor

In [2]:
def prepare_files_to_extraction(main_dir_path):
    case_vector = {}

    for case_dir in os.listdir(main_dir_path):
        if os.path.isdir(main_dir_path + '/' + case_dir):
            case_id = case_dir
            modality_dict = {}
            if os.path.isdir(main_dir_path + '/' + case_id):
                for file in os.listdir(main_dir_path + '/' + case_id):
                    case_specific_id = file[21:25]
                    if case_specific_id == '0000':
                        modality_dict['FLAIR'] = main_dir_path + '/' + case_id + '/' + file
                    elif case_specific_id == '0001':
                        modality_dict['T1'] = main_dir_path + '/' + case_id + '/' + file
                    elif case_specific_id == '0002':
                        modality_dict['T1C'] = main_dir_path + '/' + case_id + '/' + file
                    elif case_specific_id == '0003':
                        modality_dict['T2'] = main_dir_path + '/' + case_id + '/' + file
                    elif case_specific_id == 'MASK':
                        modality_dict['MASK'] = main_dir_path + '/' + case_id + '/' + file
                    else:
                        print('Wrong file %s found in case folder: %s' % (file, case_id))
                        exit()
            case_vector[case_id] = modality_dict
    return case_vector

In [3]:
def initialize_extractor():
    settings = {'binWidth': 25, 'resampledPixelSpacing': None, 'interpolator': sitk.sitkBSpline}
    extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
    return extractor

In [4]:
def extract_features(case_vector, extractor):
    feature_vector = {}

    for case_id in sorted(case_vector.keys()):
        print("Working on case: ", case_id)
        case_feature_vector = {}
        
        modality_list = case_vector[case_id]
        mask = modality_list["MASK"]
        modality_list.pop("MASK")
        
        for modality in sorted(modality_list.keys()):
            print('Calculating features for modality: ', modality)
            image = modality_list[modality]
            if image is None or mask is None:
                print('There is no image in case ' + case_id + ' modality ' + modality)
                exit()
            local_feature_vector = extractor.execute(image, mask)

            for feature_name in local_feature_vector.copy().keys():
                if "diagnostics_" in feature_name:
                    local_feature_vector.pop(feature_name)
            for feature_name in local_feature_vector.copy().keys():
                if type(local_feature_vector[feature_name]) ==  np.ndarray:
                    local_feature_vector[feature_name] = local_feature_vector[feature_name].tolist()
            case_feature_vector[modality] = local_feature_vector

        feature_vector[case_id] = case_feature_vector
    return feature_vector

In [5]:
def format_features(feature_vector, mapping_case_ids, mapping_case_grades):
    case_ids = []
    feature_ids = []
    flattened_feature_vector = {}

    for case_id in feature_vector.keys():
        case_vector = feature_vector[case_id]
        for modality_id in case_vector.keys():
            modality_vector = case_vector[modality_id]
            flattened_feature_vector[case_id + "_" + modality_id] = modality_vector
            case_ids.append(case_id + "_" + modality_id)

    case_vector = feature_vector[list(feature_vector.keys())[0]]
    modality_vector = case_vector[list(case_vector.keys())[0]]

    for feature_name in modality_vector.keys():
        feature_ids.append(feature_name)
    feature_ids.append("Grade")

    features = pd.DataFrame(index=case_ids ,columns=feature_ids)
    j = 0
    for case_id in flattened_feature_vector.keys():
        case_vector = flattened_feature_vector[case_id]
        feature_values = []

        for feature_name in case_vector.keys():
            feature_value = case_vector[feature_name]
            feature_values.append(feature_value)
            
        case_general_id = case_id[0:20]
        
        if case_general_id in mapping_case_ids:
            index = np.where(mapping_case_ids == case_general_id)
            feature_values.append(mapping_case_grades[index][0])

        for i in range(len(feature_values)):
            features.iloc[j][i] = feature_values[i]
        j += 1
        
    return features

In [6]:
name_mapping = pd.read_csv("mapping/name_mapping.csv")

case_ids = name_mapping.values[:,1]
case_grades = name_mapping.values[:,0]

In [7]:
extractor = initialize_extractor()

In [8]:
case_vector = prepare_files_to_extraction('/Users/I517596/jupyter-notebooks/final-project/data')

In [9]:
feature_vector = extract_features(case_vector, extractor)

Working on case:  BraTS20_Training_001
Calculating features for modality:  FLAIR


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Calculating features for modality:  T1


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Calculating features for modality:  T1C


...

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Calculating features for modality:  T1C


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Calculating features for modality:  T2


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [10]:
features = format_features(feature_vector, case_ids, case_grades)

In [11]:
features.to_csv("output/new_features.csv")